**Next method**

In [21]:
import sys
print(sys.executable)

c:\Users\edward_b\OneDrive - Institute for Fiscal Studies\Work\Brazil social insurance\venv\Scripts\python.exe


In [1]:
import os
print(os.getcwd())

c:\Users\edward_b\OneDrive - Institute for Fiscal Studies\Work\Brazil social insurance\scripts


***Pyautogui***

In [1]:
import pyautogui
import time
import subprocess
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import psutil
from selenium.webdriver.support.ui import Select
import matplotlib.pyplot as plt
import logging
from logging.handlers import RotatingFileHandler
import random
import requests
import re
import shutil
import os

from utils import extract_cpf, kill_chrome, cnpj_check, scrape_data, obtain_pdf, debt_collector, outstanding_payment, scrape_debt_table

In [2]:
os.chdir("C:/Users/edward_b/OneDrive - Institute for Fiscal Studies/Work/Brazil social insurance")


**Import master**

In [25]:
# path = "raw/CNPJ numbers"
# cnpj_master = pd.read_csv(f'{path}/simples.csv', sep=',', encoding='utf-8')
# cnpj_master = cnpj_master[['cnpj_basico','opcao_mei']]

# # find length of cnpj
# cnpj_master["length cnpj_basico"] = cnpj_master["cnpj_basico"].astype(str).str.len()
# pd.crosstab(cnpj_master["length cnpj_basico"], cnpj_master["opcao_mei"], margins=True, margins_name="Total")

# cnpj_master["cnpj_basico"] = cnpj_master["cnpj_basico"].astype(str)
# cnpj_master = cnpj_master[cnpj_master['opcao_mei'] == 1]
# cnpj_master.drop(columns=['length cnpj_basico'], inplace=True)

# cnpj_test = pd.read_csv(f'{path}/establishmentsPI.csv', sep=',', encoding='utf-8')
# #cnpj_test = cnpj_test[['cnpj']]
# cnpj_test["cnpj_basico"] = cnpj_test["cnpj"].astype(str).str[:8]

# cnpj_merged = pd.merge(cnpj_master, cnpj_test, left_on='cnpj_basico', right_on='cnpj_basico', how='inner')
# cnpj_merged["cnpj"] = cnpj_merged["cnpj"].astype(str)
# #cnpj_merged["cnpj"].str.len().hist()

# cnpj_merged = cnpj_merged[cnpj_merged['cnpj'].str.len() == 14]

# #Check for duplicates
# duplicates = cnpj_merged[cnpj_merged['cnpj'].duplicated(keep=False)]
# print(len(duplicates) == 0)
# cnpj_merged = cnpj_merged.drop_duplicates(subset=['cnpj'], keep='first')

# cnpj_merged.to_csv('MEI_numbers.csv', sep=',', encoding='utf-8', index=False)

*****-----------------------Load in MEI numbers and start scraping-----------------------*****

In [26]:
cnpj_merged = pd.read_csv('MEI_numbers.csv', sep=',', encoding='utf-8')

In [27]:
# proxy_list = pd.read_csv("scripts/proxies.txt", sep=',', encoding='utf-8', header=None, names=['proxy'])
# proxy_list = proxy_list["proxy"].to_list()
# proxy_list

In [28]:
# get random sample of 10
cnpj_merged = cnpj_merged.sample(n=5, random_state=10)
#convert cnpj's to a list
cnpj_list = cnpj_merged['cnpj'].tolist()
cnpj_merged.shape # 13,894 obs

(5, 3)

**Set up scraper**

In [29]:
cnpj_list = [str(i) for i in cnpj_list]
cnpj_list

['28740844000198',
 '11757216000112',
 '33962463000193',
 '31898571000119',
 '12902387000150']

In [3]:
cnpj_list = ["35184782000140"] #,"33962463000193", "31898571000119"]


In [4]:
import shutil
import os

chrome_profile_path = "C:/Temp/ChromeDebug"
if os.path.exists(chrome_profile_path):
    shutil.rmtree(chrome_profile_path)  # delete the profile folder

In [32]:
log_file = 'mei_scraper_log.log'
with open(log_file, 'w'):
    pass  # This empties the file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        RotatingFileHandler(log_file, maxBytes=5*1024*1024, backupCount=3),  # 5MB per file
        logging.StreamHandler()  # Optional: print to console too
    ]
)

In [33]:
pyautogui.position()

Point(x=1749, y=341)

In [ ]:
# ---- Define url and cnpj list ----
url = "https://www8.receita.fazenda.gov.br/SimplesNacional/Aplicacoes/ATSPO/pgmei.app/Identificacao"

timings = []
total_start_time = time.time()
master_df = pd.DataFrame()
master_debt_df = pd.DataFrame()  # DataFrame to store debt collection data


for cnpj in cnpj_list:
    #proxy = random.choice_list)
    #logging.info("Processing CNPJ:", cnpj)
    start_time = time.time()
    data = []

    chrome_profile_path = "C:/Temp/ChromeDebug"
    if os.path.exists(chrome_profile_path):
        shutil.rmtree(chrome_profile_path)  # delete the profile folder to avoid Captcha detection. If want to change where downloaded files go, open Chrome in command prompt and change the settings manually.

    try:

        # ---- Step 1: Start Chrome in remote debug mode ----
        subprocess.Popen([
            r"C:/Program Files/Google/Chrome/Application/chrome.exe",
            #f"--proxy-server={proxy}",
            "--remote-debugging-port=9222",
            "--user-data-dir=" + chrome_profile_path,
            "--start-maximized",  # or "--start-fullscreen"
            "--disable-popup-blocking",  # optional, disable for debugging only
            "--disable-extensions",
            "--no-first-run",
            "--no-default-browser-check"
        ])
        time.sleep(2)  # Give Chrome time to launch

        # ---- Step 2: Use pyautogui to interact with the site ----
        pyautogui.hotkey('ctrl', 'l')
        pyautogui.typewrite(url, interval=0.01)
        pyautogui.press('enter')  
        time.sleep(2)

        pyautogui.moveTo(x=1027, y=377, duration=1) # laptop x=722, y=391  deksptop : x=1027, y=377
        pyautogui.click()
        pyautogui.typewrite(cnpj, interval=0.1)

        pyautogui.moveTo(x=1027, y=500, duration=1) # Laptop x=722, y=514 desktop: x=1027, y=500
        pyautogui.click()
        time.sleep(2)

        ## Selenium setup
        options = Options()
        options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

        #options.add_argument(f'--proxy-server={proxy}')
        driver = webdriver.Chrome(options=options)
        wait = WebDriverWait(driver, 3)

        driver.get("https://www8.receita.fazenda.gov.br/SimplesNacional/Aplicacoes/ATSPO/pgmei.app/emissao")
        time.sleep(1.5)

        # check the cnpj on the webpage is the same as the one we are looking for
        cnpj_check(driver, cnpj)

        # First try: Bootstrap-styled dropdown
        try:
            dropdown_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-id="anoCalendarioSelect"]')))
            dropdown_button.click()
            time.sleep(1)

            year_elements = wait.until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, ".dropdown-menu.inner li a span.text")
            ))
            enabled_years = [el.text.strip() for el in year_elements if el.text.strip()]
            # remove elements from the list that contain "Não optante"
            enabled_years = [year for year in enabled_years if "Não optante" not in year]

            # Raise an exception if no enabled years are found
            if not enabled_years:
                raise ValueError("No enabled years found in the dropdown menu.")

            print("Bootstrap dropdown enabled years for CNPJ ", cnpj , ":", enabled_years)
            use_bootstrap = True
            

        except Exception as e:
            print("Bootstrap dropdown failed, falling back to native <select> method.")
            # Try native <select>
            select_element = wait.until(EC.presence_of_element_located((By.ID, "anoCalendarioSelect")))
            dropdown = Select(select_element)
            enabled_years = [o.text.strip() for o in dropdown.options if o.text.strip()]
            enabled_years = [year for year in enabled_years if "Não optante" not in year]
            print("Native <select> enabled years for CNPJ ", cnpj,":", enabled_years)
            use_bootstrap = False

        print("scraping years", enabled_years)
        #enabled_years = [str(max(enabled_years))] # to just scrape the most recent year
        enabled_years.insert(0, "2010")  #add a year to the start of the list
        
        obtained_pdf = 0
        for index, year in enumerate(enabled_years):
            try:
                if use_bootstrap:
                    dropdown_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-id="anoCalendarioSelect"]')))
                    driver.execute_script("arguments[0].click();", dropdown_button)
                    time.sleep(1.5)  # allow dropdown to render
                    
                    print("Clicking on year:", year)
                    year_option = wait.until(EC.element_to_be_clickable(
                        (By.XPATH, f"//span[@class='text' and normalize-space(text())='{year}']")
                    ))
                    time.sleep(2)
                    driver.execute_script("arguments[0].click();", year_option)
                    #ActionChains(driver).move_to_element(year_option).click().perform()
                    print(f"Selected (Bootstrap) year: {year}")
                else:
                    dropdown = Select(driver.find_element(By.ID, "anoCalendarioSelect"))
                    dropdown.select_by_visible_text(year)
                    print(f"Selected (native) year: {year}")

                ok_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
                ok_button.click()
                time.sleep(2)
                
                # check if table is present
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                table = soup.find('table', class_='table table-hover table-condensed emissao is-detailed')
                if not table:
                    print(f"No table found for {cnpj} in year {year}. Skipping the rest.")
                    # Mark all remaining years as not found
                    for remaining_year in enabled_years[index:]:
                        data.append({ 
                            'cnpj': cnpj,
                            'Período de Apuração': remaining_year,
                            'data_found': False
                        })
                        # Convert missing data to DataFrame and append to master_df
                    missing_df = pd.DataFrame(data)
                    master_df = pd.concat([master_df, missing_df], ignore_index=True)
                    break  # Exit the year loop
               
               #check if the table is subject to debt collection
                is_debt_collector = debt_collector(soup)
                
                # Scrape the main table
                new_data = scrape_data(cnpj, year, soup, table)
                master_df = pd.concat([master_df, new_data], ignore_index=True)
                
                if is_debt_collector:
                # Also scrape the debt collection table
                    print(f"Debt collection table found in year {year}.")
                    debt_data = scrape_debt_table(cnpj,soup)
                    master_debt_df = pd.concat([master_debt_df, debt_data], ignore_index=True)

                # Check if outstanding payments exist
                outstanding_payments = outstanding_payment(new_data)

                # Try to obtain PDF if not in debt collection
                if not is_debt_collector and outstanding_payments and not obtained_pdf:
                    print(f"Outstanding payments found in year {year}, period {outstanding_payments}, attempting to obtain PDF.")
                    obtain_pdf(driver,wait, outstanding_payments)
                    driver.back()
                    obtained_pdf = 1
                    
                
                driver.back()
                time.sleep(2)

            except Exception as e:
                print(f"Error with year {year}:", e)

    except Exception as outer_error:
        print(f"Fatal error with CNPJ {cnpj}:", outer_error)

    finally:
        try:
            driver.quit()
        except:
            pass
        kill_chrome()
        end_time = time.time() 
        elapsed = end_time - start_time
        timings.append(elapsed)
        total_elapsed = time.time() - total_start_time
        average_elapsed = sum(timings) / len(timings)
        
        logging.info(f"Finished CNPJ: {cnpj} in {elapsed:.2f} seconds\n")
        logging.info(f"Average time per CNPJ: {average_elapsed:.2f} seconds")
        logging.info(f"Total time elapsed: {total_elapsed:.2f} seconds\n")
        time.sleep(2)

**Order and Export**

In [21]:


# replace quotas with 0 if it is NaN
master_df['Quotas'] = master_df['Quotas'].fillna(0).astype(int)
master_df["year"] = master_df["Período de Apuração"].str.extract(r'(\d{4})').astype(int)
#Extract month as text before '/'
master_df["month"] = master_df["Período de Apuração"].str.extract(r'(\w+)')

# Convert month to numerical value
month_mapping = {
    'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4,
    'maio': 5, 'junho': 6, 'julho': 7, 'agosto': 8,
    'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12
}
master_df['month'] = master_df['month'].str.lower().map(month_mapping)

master_df = master_df[['cnpj', 'Período de Apuração','year','month', 'Apurado', 'Situação', 'Benefício INSS',
         'Quotas', 'Principal', 'Multa', 'Juros', 'Total',
         'Data de Vencimento', 'Data de Acolhimento', 'data_found']]

master_df = master_df.sort_values(by=['cnpj', 'year', 'month'])
# pd.DataFrame(data).to_csv('MEI_data.csv', sep=',', encoding='utf-8', index=False)

**Import PDFs and build database of CNPJ and CPF**

In [ ]:
## import pdfs
pdf_path = os.path.abspath("scraped data/pdfs") # move downloaded pdfs to this directory

#load up all pdfs in the pdf_path directory
mei_pdf = [os.path.join(pdf_path, f) for f in os.listdir(pdf_path) if f.startswith('DAS-PGMEI')]

#extract cpf from pdfs
cnpj_cpf = [extract_cpf(i) for i in mei_pdf]
cnpj_cpf = pd.DataFrame(cnpj_cpf, columns=['cnpj', 'cpf'])
cnpj_cpf

,cnpj,cpf
0,31898571000119,80283896272
1,33962463000193,32563388287
2,35184782000140,00266193269
